<a href="https://colab.research.google.com/github/dinuka-rp/nerdy-panda/blob/main/Prototype/Data%20Collection/data_mining_nfts_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Mining NFT Data

## 2.1. NFT Data Collection 

OpenSea provides different APIs for fetching NFTs data. We will be using 2 of them:
- [OpenSea Assets API](https://docs.opensea.io/reference#getting-assets)
- [OpenSea Events API](https://docs.opensea.io/reference#retrieving-asset-events)

```

### 2.1.2. Fetching Data from OpenSea APIs

We will use Python to fetch data from 2 OpenSea APIs:

* [OpenSea Assets API](https://docs.opensea.io/reference#getting-assets): We will be using this API to retrieve the following data about each NFT:

    - The Meebit's Id.
    - The creator's username and address. The creator is the person who minted the Meebit to the blockchain.
    - The owner's username and address.
    - The Meebit's traits.
    - The number of times the Meebit has been sold.

* [OpenSea Events API](https://docs.opensea.io/reference#retrieving-asset-events): We will be using this API to retrieve the following data about each Meebit’s sale transaction:

    - The Meebit's Id.
    - The seller's username and address.
    - The buyer's username and address.
    - The sale's timestamp.
    - The sale transaction's hash.
    - The price.
    - The token used for the sale.
    - The price in USD.
    - If the sale is a bundle of Meebits.

I prepared [2 python functions](https://github.com/adilmoujahid/data-mining-nfts/blob/main/helpers.py) ```parse_meebit_data(meebit_dict)``` and ```parse_sale_data(sale_dict)``` for parsing the data that we’ll fetch from the OpenSea APIs. 

Next, from a Jupyter notebook, we import the Python libaries that we'll use for the analysis.

## Mount Google Drive



In [1]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
# %matplotlib inline

In [2]:
def parse_nft_asset_data(nft_dict):
    
    nft_id = nft_dict['token_id']
    
    try:
        name = nft_dict['name']
    except:
        name = None

    try:
        asset_description = nft_dict['description']
    except:
        asset_description = None

    try:
        collection_name = nft_dict['collection']['name']
    except:
        collection_name = None
    try:
        collection_description = nft_dict['collection']['description']
    except:
        collection_description = None
    try:
        collection_slug = nft_dict['collection']['slug']
    except:
        collection_slug = None

    try:
        creator_username = nft_dict['creator']['user']['username']
    except:
        creator_username = None
    try:
        creator_address = nft_dict['creator']['address']
    except:
        creator_address = None
    
    try:
        owner_username = nft_dict['owner']['user']['username']

    except:
        owner_username = None
    
    owner_address = nft_dict['owner']['address']
    if owner_address = "0x0000000000000000000000000000000000000000":
      owner_address = None

    
    traits = nft_dict['traits']
    num_sales = int(nft_dict['num_sales'])

    # this data may be valuable for a price prediction & to identify who owned the nft before and the transacted amount - might be able to get all these from the events api itself. 
    try:
        last_sale = nft_dict['last_sale']
    except:
        last_sale = None

    try:
        top_bid = nft_dict['top_bid']
    except:
        top_bid = None

    result = {'nft_id': nft_id,
              'name': name,
              'asset_description': asset_description,
              'collection_name': collection_name,
              'collection_description': collection_description,
              'collection_slug': collection_slug,
              'creator_username': creator_username,
              'creator_address': creator_address,
              'owner_username': owner_username,
              'owner_address': owner_address,
              'traits': traits,
              'num_sales': num_sales,
              'last_sale': last_sale,
              'top_bid': top_bid}
    
    return result


def parse_sale_data(sale_dict):
    
    is_bundle = False

    if sale_dict['asset'] != None:
        nft_id = sale_dict['asset']['token_id']
    elif sale_dict['asset_bundle'] != None:
        nft_id = [asset['token_id'] for asset in sale_dict['asset_bundle']['assets']]
        is_bundle = True
    
    
    seller_address = sale_dict['seller']['address']
    buyer_address = sale_dict['winner_account']['address']
    
    try:
        seller_username = sale_dict['seller']['user']['username']
    except:
        seller_username = None    
    try:
        buyer_username = sale_dict['winner_account']['user']['username']
    except:
        buyer_username = None
    
    timestamp = sale_dict['transaction']['timestamp']
    total_price = float(sale_dict['total_price'])
    payment_token = sale_dict['payment_token']['symbol']
    usd_price = float(sale_dict['payment_token']['usd_price'])
    transaction_hash = sale_dict['transaction']['transaction_hash']
    

    result = {'is_bundle': is_bundle,
              'nft_id': nft_id,
              'seller_address': seller_address,
              'buyer_address': buyer_address,
              'buyer_username': buyer_username,
              'seller_username':seller_username,
              'timestamp': timestamp,
              'total_price': total_price, 
              'payment_token': payment_token,
              'usd_price': usd_price,
              'transaction_hash': transaction_hash}
    
    return result

In [7]:
# from helpers import parse_meebit_data, parse_sale_data
import requests
import pandas as pd
import os

In [ ]:
# import matplotlib
# import matplotlib.pyplot as plt

In [ ]:
# plt.style.use('ggplot')

## Getting NFT Assets Data

The source code below collects assets data about the 20,000 NFTs. The API has a limit of 50 items per call, and therefore we need to create a loop with 100 iterations to collect the required amount of NFT data.

In [8]:
assets_file_path = "/content/drive/MyDrive/Datasets/FYP/collected-nft-assets"

In [19]:
url = "https://api.opensea.io/api/v1/assets"

for i in range(1, 100):
    offset= 50 * (i - 1)

    querystring = {"order_direction":"desc",
                   "offset":offset,
                   "limit":"50",
                   #"collection":"nike"
                   }
    response = requests.request("GET", url, params=querystring)
    
    print(response.json()['assets'])
    if response.status_code != 200:
        print('error')
        break
    
    #Getting NFT data
    nfts = response.json()['assets']
    #Parsing NFT data
    parsed_nft_assets = [parse_nft_asset_data(nft) for nft in nfts]

    # convert json to pandas dataframe
    nft_assets_df = pd.DataFrame(parsed_nft_assets)

    # save dataframe to csv & save csv to Google Drive
    nft_assets_df.to_csv(assets_file_path, mode='a', sep='\t', index=False, header=(not os.path.exists(assets_file_path)), encoding='utf-8')

[{'id': 124654853, 'token_id': '23977370117191917724225468807392124890950272782277708039151276164591327903745', 'num_sales': 0, 'background_color': None, 'image_url': 'https://lh3.googleusercontent.com/PTvRTHI9atJiDfChrTTzlP4LOwEBxwjPK60-437Nn8rA38NY4rOYDjP03WnK8xhsnskF2eltf8C5-ypOu7AyyPfgLWzhnQu9T-0kSw', 'image_preview_url': 'https://lh3.googleusercontent.com/PTvRTHI9atJiDfChrTTzlP4LOwEBxwjPK60-437Nn8rA38NY4rOYDjP03WnK8xhsnskF2eltf8C5-ypOu7AyyPfgLWzhnQu9T-0kSw=s250', 'image_thumbnail_url': 'https://lh3.googleusercontent.com/PTvRTHI9atJiDfChrTTzlP4LOwEBxwjPK60-437Nn8rA38NY4rOYDjP03WnK8xhsnskF2eltf8C5-ypOu7AyyPfgLWzhnQu9T-0kSw=s128', 'image_original_url': None, 'animation_url': None, 'animation_original_url': None, 'name': 'DD2153', 'description': 'DD2153\n', 'external_link': None, 'asset_contract': {'address': '0x495f947276749ce646f68ac8c248420045cb7b5e', 'asset_contract_type': 'semi-fungible', 'created_date': '2020-12-02T17:40:53.232025', 'name': 'OpenSea Collection', 'nft_version': N

### Read CSV - check collected NFT data

In [26]:
nft_assets_df = pd.read_csv(assets_file_path, sep='\t')

nft_assets_df.tail()

,nft_id,name,asset_description,collection_name,collection_description,collection_slug,creator_username,creator_address,owner_username,owner_address,traits,num_sales,last_sale,top_bid
145,5943720334399124486105950946209096681388229894...,Lucky Binary NFT Number #721223,NaN,Lucky Binary NFT Number,Lucky Binary NFT Number ☘️☘️☘️\n\nChoose your ...,lucky-binary-nft-number,iDaniel,0x836843d23b5ea5c49c87a5b6bd4f9729964f93b4,NullAddress,0x0000000000000000000000000000000000000000,[],0,NaN,NaN
146,11677,Fisherman #11677,Some description content...,Whales.Game,"A refined, highly efficient minigame with addi...",official-whales-game,NaN,0x7a5e9ab89a7e70e14e731aea2a212b7fd245fc82,BewareTheTyrannyOfknowledge,0xb95b8106b30c794b1eed3c5eeaf6a02cf82c8446,"[{'trait_type': 'Type', 'value': 'Fisherman', ...",0,NaN,NaN
147,2300538991603141949549706883158928800990063278...,My Little Piggie Club #433,My Little Piggie Club #433\n\nCollection of 10...,My Little Piggie Club,NaN,my-little-piggie,NFT_Collector_Investor,0x32dc96d9c14bb125d80e542eb44a4c9252a16cee,NullAddress,0x0000000000000000000000000000000000000000,"[{'trait_type': 'Accessory', 'value': 'Hook', ...",0,NaN,NaN
148,7494752752623448078296476341828787674872100358...,Pinky Pig #43,NaN,Pinky Pigs,NaN,pinky-pigs,narumiobata,0xa5b2cc1cca8e8b6650e90b7b3f46a21a2cd5fe82,NullAddress,0x0000000000000000000000000000000000000000,"[{'trait_type': 'ACCESSORY', 'value': 'Cigaret...",0,NaN,NaN
149,11676,Fisherman #11676,Some description content...,Whales.Game,"A refined, highly efficient minigame with addi...",official-whales-game,NaN,0x7a5e9ab89a7e70e14e731aea2a212b7fd245fc82,BewareTheTyrannyOfknowledge,0xb95b8106b30c794b1eed3c5eeaf6a02cf82c8446,"[{'trait_type': 'Head', 'value': 'Dreadlocks',...",0,NaN,NaN


## Clean Asset Data

*   Remove NFTs that have no text that can be worked with at all (descriptions/ sensible names - nouns, non-stop words,etc.) - can have this check when parsing as well (check later)


## Getting NFT sales transactions data

The source code below collects all sale transactions data. The API has a limit of 50 items per call, and therefore we need to create a loop to collect all sale transactions data.

In [ ]:
url = "https://api.opensea.io/api/v1/events"

for i in range(1, 100):
    offset= 50 * (i - 1)

    querystring = {
                   "event_type":"successful",
                   "only_opensea":"true",
                   "offset":offset,
                   "limit":"50",
                   }

    headers = {"Accept": "application/json"}

    response = requests.request("GET", url, headers=headers, params=querystring)

    
    print(i)
    if response.status_code != 200:
        print('error')
        break
    
    #Getting meebits sales data
    meebit_sales = response.json()['asset_events']

    if meebit_sales == []:
        break
    
    #Parsing meebits sales data
    parsed_meebit_sales = [parse_sale_data(sale) for sale in meebit_sales]

  # save this in a separate CSV file and merge later



0 

ServerSelectionTimeoutError: ignored

## 2.2. Analyzing Meebits Data and Meebits Sale Transactions

Now that we have the data in MongoDB, we can start analyzing it. 

### 2.2.1. Reading the Data
We start by reading both the assets data and the transactions data into 2 Pandas DataFrames.

In [ ]:
meebits = meebits_collection.find()
meebits_df = pd.DataFrame(meebits)

meebit_sales = sales_collection.find()
meebit_sales_df = pd.DataFrame(meebit_sales)

In [ ]:
print("The database has information about %d Meebits." % len(meebits_df))
print("The database has information about %d Meebits sale transactions." % len(meebit_sales_df))

### 2.2.2. Getting Top 10 Meebits Creators

In [ ]:
creators = []
for creator_address in meebits_df['creator_address'].value_counts().index[:10]:
    creator_data = {}
    creator_data['creator_address'] = creator_address
    creator_data['creator_username'] = meebits_df[meebits_df['creator_address'] == creator_address]['creator_username'].iloc[0]
    creator_data['number_meebits'] = len(meebits_df[meebits_df['creator_address'] == creator_address])
    creators.append(creator_data)

pd.DataFrame(creators)

In [ ]:
#### Getting total number of Meebit Creators and Owners.
print("There are %d unique Meebit creators." % len(meebits_df['creator_address'].unique()))
print("There are %d unique Meebit owners." % len(meebits_df['owner_address'].unique()))

### 2.2.3. Getting Stats about Bundle/Single Sales and Types of Payment

In [ ]:
meebit_sales_df['is_bundle'].value_counts()

In [ ]:
meebit_sales_df[meebit_sales_df['is_bundle'] == False]['payment_token'].value_counts()

### 2.2.4. Filering Sale Transactions and Adding New Features

To make the analysis easier, we will only focus on single sales done in ETH or WETH.

In [ ]:
meebit_sales_df = meebit_sales_df[(meebit_sales_df['payment_token'] != 'USDC') & (meebit_sales_df['is_bundle'] == False)].copy()

Next, we do some data cleaning and we add a new feature

In [ ]:
# Parsing dates
meebit_sales_df['timestamp'] = pd.to_datetime(meebit_sales_df['timestamp'])
# Converting sales price from WEI to ETH
meebit_sales_df['total_price'] = meebit_sales_df['total_price']/10.**18
# Calculating the sale prices in USD
meebit_sales_df['total_price_usd'] = meebit_sales_df['total_price'] * meebit_sales_df['usd_price']

### 2.2.5. Meebits Sales Timelines

#### Total Number of Sales per Day

In [ ]:
data = meebit_sales_df[['timestamp', 'total_price']].resample('D', on='timestamp').count()['total_price']
ax = data.plot.bar(figsize=(18, 6))

ax.set_alpha(0.8)
ax.set_title("Number of Meebits Sales per Day", fontsize=18)
ax.set_ylabel("Number of Meebits Sales", fontsize=18)

#https://github.com/pandas-dev/pandas/issues/1918
plt.gca().xaxis.set_major_formatter(plt.FixedFormatter(data.index.to_series().dt.strftime("%d %b %Y")))

#https://robertmitchellv.com/blog-bar-chart-annotations-pandas-mpl.html
for i in ax.patches:
    # get_x pulls left or right; get_height pushes up or down
    ax.text(i.get_x(), i.get_height()+40, \
            str(round((i.get_height()), 2)), fontsize=11, color='dimgrey',
                rotation=45)

#### Total Sales per Day in ETH

In [ ]:
data = meebit_sales_df[['timestamp', 'total_price']].resample('D', on='timestamp').sum()['total_price']
ax = data.plot(figsize=(18,6), color="red", linewidth=1, marker='o', markerfacecolor='grey', markeredgewidth=0)

ax.set_alpha(0.8)
ax.set_title("Timeline of Total Meebit Sales in ETH", fontsize=18)
ax.set_ylabel("Sales in ETH", fontsize=18);

dates = list(data.index)
values = list(data.values)

for i, j in zip(dates, values):
    ax.annotate(s="{:.0f}".format(j), xy=(i, j+200), rotation=45)

#### Total Sales per day in USD

In [ ]:
data = meebit_sales_df[['timestamp', 'total_price_usd']].resample('D', on='timestamp').sum()['total_price_usd']
ax = data.plot(figsize=(18,6), color="red", linewidth=1, marker='o', markerfacecolor='grey', markeredgewidth=0)

ax.set_alpha(0.8)
ax.set_title("Timeline of Total Meebit Sales in Million USD", fontsize=18)
ax.set_ylabel("Sales in Million USD", fontsize=18);

dates = list(data.index)
values = list(data.values)

for i, j in zip(dates, values):
    ax.annotate(s="{:.2f}".format(j/10.**6), xy=(i, j), rotation=45)

### 2.2.6. Meebits Prices Timelines 

#### Average Meebit Price per Day in ETH 

In [ ]:
data = meebit_sales_df[['timestamp', 'total_price']].resample('D', on='timestamp').mean()['total_price']
ax = data.plot(figsize=(18,6), color="green", linewidth=1, marker='o', markerfacecolor='grey', markeredgewidth=0)

ax.set_alpha(0.8)
ax.set_title("Timeline of Average Meebit Price in ETH", fontsize=18)
ax.set_ylabel("Average Price in ETH", fontsize=18);
#ax.annotate(s='sdsdsds', xy=(1, 1))

dates = list(data.index)
values = list(data.values)

for i, j in zip(dates, values):
    ax.annotate(s="{:.2f}".format(j), xy=(i, j+.2), rotation=45)

#### Floor Meebit Price per Day in ETH

In [ ]:
data = meebit_sales_df[['timestamp', 'total_price']].resample('D', on='timestamp').min()['total_price']
ax = data.plot(figsize=(18,6), color="orange", linewidth=1, marker='o', markerfacecolor='grey', markeredgewidth=0)

ax.set_alpha(0.8)
ax.set_title("Timeline of Floor Meebit Price in ETH", fontsize=18)
ax.set_ylabel("Floor Price in ETH", fontsize=18);

dates = list(data.index)
values = list(data.values)

for d, v in zip(dates, values):
    ax.annotate(s="{:.2f}".format(v), xy=(d, v), rotation=45)

#### Max Meebit Price per Day in ETH

In [ ]:
data = meebit_sales_df[['timestamp', 'total_price']].resample('D', on='timestamp').max()['total_price']
ax = data.plot(figsize=(18,6), color="red", linewidth=1, marker='o', markerfacecolor='grey', markeredgewidth=0)

ax.set_alpha(0.8)
ax.set_title("Timeline of Max Meebit Price in ETH", fontsize=18)
ax.set_ylabel("Max Price in ETH", fontsize=18);

dates = list(data.index)
values = list(data.values)

for i, j in zip(dates, values):
    ax.annotate(s="{:.0f}".format(j), xy=(i, j+30), rotation=45)

### 2.2.6. Analyzing Meebits' Sellers and Buyers

In [ ]:
print("There are %d unique Meebit sellers." % len(meebit_sales_df['seller_address'].unique()))
print("There are %d unique Meebit buyers." % len(meebit_sales_df['buyer_address'].unique()))

#### Getting Top 10 Meebits Buyers

In [ ]:
buyers = []
for buyer_address in meebit_sales_df['buyer_address'].value_counts().index[:10]:
    buyer_data = {}
    buyer_data['buyer_address'] = buyer_address
    buyer_data['buyer_username'] = meebit_sales_df[meebit_sales_df['buyer_address'] == buyer_address]['buyer_username'].iloc[0]
    buyer_data['number_buys'] = len(meebit_sales_df[meebit_sales_df['buyer_address'] == buyer_address])
    buyer_data['min_price'] = meebit_sales_df[meebit_sales_df['buyer_address'] == buyer_address]['total_price'].min()
    buyer_data['max_price'] = meebit_sales_df[meebit_sales_df['buyer_address'] == buyer_address]['total_price'].max()
    buyer_data['mean_price'] = meebit_sales_df[meebit_sales_df['buyer_address'] == buyer_address]['total_price'].mean()
    buyers.append(buyer_data)
    
pd.DataFrame(buyers)

#### Getting Top 10 Meebits Sellers

In [ ]:
sellers = []
for seller_address in meebit_sales_df['seller_address'].value_counts().index[:10]:
    seller_data = {}
    seller_data['seller_address'] = seller_address
    seller_data['seller_username'] = meebit_sales_df[meebit_sales_df['seller_address'] == seller_address]['seller_username'].iloc[0]
    seller_data['number_sales'] = len(meebit_sales_df[meebit_sales_df['seller_address'] == seller_address])
    seller_data['min_price'] = meebit_sales_df[meebit_sales_df['seller_address'] == seller_address]['total_price'].min()
    seller_data['max_price'] = meebit_sales_df[meebit_sales_df['seller_address'] == seller_address]['total_price'].max()
    seller_data['mean_price'] = meebit_sales_df[meebit_sales_df['seller_address'] == seller_address]['total_price'].mean()
    sellers.append(seller_data)
    
pd.DataFrame(sellers)

#### Intersection of Top 10 Buyers and Top 10 sellers

In [ ]:
top_10_buyers = meebit_sales_df['buyer_address'].value_counts().index[:10]
top_10_sellers = meebit_sales_df['seller_address'].value_counts().index[:10]
print(list(set(top_10_buyers) & set(top_10_sellers)))

#### Getting Number of Sales between same Buyers and Sellers 

In [ ]:
(meebit_sales_df['seller_address'] + meebit_sales_df['buyer_address']).value_counts().value_counts()